In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, precision_recall_fscore_support, precision_recall_curve
from imblearn.over_sampling import SMOTE  # Import SMOTE
import matplotlib.pyplot as plt 

In [2]:
df = pd.read_csv('predictive_maintenance_dataset.csv')

In [3]:
df.head()

,date,device,failure,metric1,metric2,metric3,metric4,metric5,metric6,metric7,metric8,metric9
0,1/1/2015,S1F01085,0,215630672,55,0,52,6,407438,0,0,7
1,1/1/2015,S1F0166B,0,61370680,0,3,0,6,403174,0,0,0
2,1/1/2015,S1F01E6Y,0,173295968,0,0,0,12,237394,0,0,0
3,1/1/2015,S1F01JE0,0,79694024,0,0,0,6,410186,0,0,0
4,1/1/2015,S1F01R2B,0,135970480,0,0,0,15,313173,0,0,3


In [4]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124494 entries, 0 to 124493
Data columns (total 12 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   date     124494 non-null  object
 1   device   124494 non-null  object
 2   failure  124494 non-null  int64 
 3   metric1  124494 non-null  int64 
 4   metric2  124494 non-null  int64 
 5   metric3  124494 non-null  int64 
 6   metric4  124494 non-null  int64 
 7   metric5  124494 non-null  int64 
 8   metric6  124494 non-null  int64 
 9   metric7  124494 non-null  int64 
 10  metric8  124494 non-null  int64 
 11  metric9  124494 non-null  int64 
dtypes: int64(10), object(2)
memory usage: 11.4+ MB


In [4]:
df['device_type'] = df['device'].str[:2]

# One-hot encoding for device type
df = pd.get_dummies(df, columns=['device_type'])

In [5]:
X = df[['metric1', 'metric2', 'metric3', 'metric4', 'metric5', 'metric6', 'metric7', 'metric8', 'metric9'] + list(df.filter(like='device_type').columns)]
y = df['failure']

In [6]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [7]:
# Apply SMOTE to balance the training set
smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

In [8]:
from collections import Counter
# Verify balance
print("Original dataset shape:", Counter(y))
print("Resampled dataset shape:", Counter(y_train_balanced))


Original dataset shape: Counter({0: 124388, 1: 106})
Resampled dataset shape: Counter({0: 99507, 1: 99507})


In [9]:
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20],
    'min_samples_split': [2, 5]
}


In [10]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=42)

# Set up Randomized Search
random_search = RandomizedSearchCV(estimator=rf, param_distributions=param_grid, n_iter=10, cv=3, scoring='roc_auc', n_jobs=-1, random_state=42)

# Fit the model
random_search.fit(X_train_balanced, y_train_balanced)

# Best parameters
print(random_search.best_params_)

NameError: name 'RandomizedSearchCV' is not defined

In [11]:
rf_best = random_search.best_estimator_
rf_best.fit(X_train_balanced, y_train_balanced)
y_pred = rf_best.predict(X_test)

NameError: name 'random_search' is not defined

In [ ]:
# Evaluate the model
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("Classification Report:")
print(classification_report(y_test, y_pred))

auc_score = roc_auc_score(y_test, y_pred)
print("AUC-ROC Score:", auc_score)

precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='binary')
print(f"Precision: {precision}, Recall: {recall}, F1-Score: {f1}")


In [ ]:

# Feature importance
importances = rf_best.feature_importances_
feature_names = X.columns
feature_importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': importances})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)
print(feature_importance_df)

# Precision-Recall curve
precision, recall, _ = precision_recall_curve(y_test, y_pred)
plt.plot(recall, precision)
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall curve')
plt.show()
